In [41]:
import pandas as pd
import numpy as np

from konlpy.tag import Okt; okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

from gensim.models.word2vec import Word2Vec

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('신문기사모음(9.17)(utf8).csv')
text = data.body

In [3]:
for i, document in enumerate(text):

    # okt = Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation', 'Foreign', 'Suffix' ,'URL', 'Conjunction']: #조사, 어미, 구두점, 외국어/기호, url 제외 
            clean_words.append(word[0])
    # print(clean_words) 
    document = ' '.join(clean_words)
    # print(document) 
    text[i] = document

print(text[0], text[1])

C:\Users\admin\anaconda3\envs\konlp\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


NHN 빅풋 한게임 20년 웹 보드 게임 노하우 접목 스포츠 예측 게임 한게임 승부 예측 출시 하다 11일 밝히다 한게임 승부 예 측은 게임 내 재화 활용 하다 스포츠 경기 결과 예측 하다 보다 게임 실제 전세계 스포츠 경기 데이터 바탕 결과 맞다 힐 경우 배당률 따르다 게임 멀다 제공 다 특히 NHN 빅풋 한게임 웹 보드 게임 개발 및 서비스 노하우 바탕 차별 되다 스포츠 예측 게임 선보이다 먼저 축구 야구 농구 등 대중 인기 스포츠 외 아이스하키 테니스 탁구 등 50 종 이상 방대하다 종목 제공 하다 선택 넓히다 국내 프로 스포츠 아니다 잉글랜드 프리미어리그 EPL 메이저리그 MLB 미국 프로농구 NBA 오다 같다 글로벌 인기 리그 모두 포함 되다 스포츠 예측 항목 승무 패 핸디캡 언 더 오버 포함 하다 120 여종 달 다 이용자 경기 당 결과 예측 다양하다 항목 선택 하다 전략 게임 즐기다 수 있다 또한 이용자 성향 맞추다 즐기다 수 있다 콘텐츠 마련 하다 크로스 하다 경기 내 여러 항목 조합 하다 수 있다 높다 수준 예측 가능하다 스페셜 전반전 혹은 1 쿼터 등 결과 제공 하다 빠르다 호흡 게임 즐기다 수 있다 라이브 실제 경기 시작 되다 후 상황 지켜보다 실시간 예측 하다 수 있다 특별하다 방식 이 승부 예측 게임 처음 접 하다 이용자 쉬다 즐기다 수 있다 직관 깔끔하다 UI 를 적용 하다 북마크 활용 하다 주요 경기 추천 Hot League 관심 경기 저장 My Favorite 과 같다 기능 제공 하다 이용자 편의 높이다 경 기별 전력 분석 토론 등 통해 경기 결과 예측 도움 주다 한게임 승부 예 측은 별도 설치 과정 없이 웹페이지 바로 게임 즐기다 수 있다 NHN 은 올해 3분 연결 기준 매출 4194억원 영업 이익 274억원 기록 하다 13일 밝히다 이다 지난해 같다 기간 각각 19.2% 24.5% 늘어나다 것 3분 실적 결제 커머스 사업 이끌다 코로나 19 여파 비대 시장 확장 되다 사업 성장하다 매출 부문 별로 살펴보다 결제 광고 매출 전 년

In [48]:
# tokenize 
okt=Okt()
tokens = []
stopwords = ['하다', '되다', '있다', '들다', '들', '기자', '오다', '돼다', '년']
# stopwords = ['기자', '년']


'''각 기사별로 토큰들 나눠서 리스트로 만들기'''
# [ [1번 기사의 토큰들], [2번 기사의 토큰들], [3번 기사의 토큰들]... [n번 기사의 토큰들] ]
for line in text:
    tokens.append(okt.morphs(line))

# noun -> 각 기사별 명사 토큰 list를 모아놓은 list   (list를 모은 list)


# 새로운 dict tokens_processed 생성 
# tokens_processed = {}
whole = []


# 기사 하나하나 마다 [토큰 리스트]에 담긴 한글자짜리 토큰 제거
temp_index = 0
for lst in tokens:
    ''' 여기서 각 기사 한줄 한줄씩 처리함 '''    
    

    # 딕셔너리에 담을 토큰들 모아놓는 리스트....
    temp_words_list = []
    

    for wd in lst:
        ''' 여기서 기사별 [토큰list] 속 단어(토큰)들을 처리함 '''
        
        if len(wd) < 2:
            # 한글자 짜리면 무시하고 지나침
            continue
            
        if wd in stopwords:
            # 불용어에 해당하는 단어면 무시하고 지나침
            continue
        
        # 둘다 해당하지 않으면 임시 리스트에 담기
        temp_words_list.append(wd)
    
    
    # 필터링한 데이터 저장
    # tokens_processed[temp_index] = temp_words_list
    whole.append(temp_words_list)
    temp_index +=1

    ''' 
      0 : [ 기사1 토큰리스트 ]
      1 : [ 기사2 토큰리스트 ]
      2 : [ 기사3 토큰리스트 ]
    ...
    122 : [ 기사122 토큰리스트 ]
    
    (temp_index)    :  (temp_wors_list) 
    이런 형식으로 dictionary(tokens_processed)에 저장
    
    '''
    
# tokens_processed
# whole[:2]

## 동시출현 기반 연관어 분석
#### 같은 문맥에서 중복되는 단어들은 하나로 처리

In [15]:
count = {}   #동시출현 빈도가 저장될 dict
for line in whole:
    words = list(set(line))   #단어별로 분리한 것을 set에 넣어 중복 제거하고, 다시 list로 변경
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue   #같은 단어의 경우는 세지 않음
            elif a > b:   #A, B와 B, A가 다르게 세어지는것을 막기 위해 항상 a < b로 순서 고정
                count[b, a] = count.get((b, a), 0) + 1   #키가 이미 딕셔너리에 있는지 확인하고 키가 없으면 기본값으로 설정
            else:
                count[a, b] = count.get((a, b), 0) + 1   

In [21]:
# 데이터프레임으로 예쁘게 만들기
df = pd.DataFrame.from_dict(count, orient='index')

# [term1, term2, freq] 리스트 생성
list1 = []
for i in range(len(df)):
    list1.append([df.index[i][0], df.index[i][1], df[0][i]])

df2 = pd.DataFrame(list1, columns=['term1', 'term2', 'freq'])   #헤더 만들어서 데이터프레임 생성
df3 = df2.sort_values(by=['freq'], ascending=False)   #frequency 큰 순으로 정렬
df3 = df3.reset_index(drop=True) #index 다시 만들기

df3.head(10)

,term1,term2,freq
0,게임,보드,94
1,게임,매출,68
2,매출,보드,67
3,보드,서비스,65
4,게임,서비스,65
5,보드,출시,63
6,게임,출시,63
7,보드,시장,58
8,게임,시장,58
9,게임,모바일,57


## 통계적 가중치 기반 연관어 분석
#### 유사도 구하기 위해 단어마다 가중치 할당
- 출현 빈도, tf-idf, tf-icf, cosine similarity, jaccard similarity, overlap imilarity

In [30]:
vec = TfidfVectorizer()
vector_lines = vec.fit_transform(text) # tfidf 가중치 할당
A = vector_lines.toarray()
A.shape

(95, 3982)

In [31]:
A = A.transpose() # 문서-단어 매트릭스를 단어-문서 매트릭스로 전치 (문서 간 유사도 아닌 단어 간 유사도 구하기 위해)
A.shape

(3982, 95)

In [32]:
A_sparse = sparse.csr_matrix(A)
similarities_sparse = cosine_similarity(A_sparse, dense_output=False) # cosine similarity 계산
list(similarities_sparse.todok().items())[35000:35010]

[((993, 76), 0.17626261703135165),
 ((997, 76), 0.12543951030650058),
 ((998, 76), 0.18537972361450689),
 ((1001, 76), 0.1563247253670177),
 ((1002, 76), 0.7766864002041666),
 ((1006, 76), 0.04290430643798979),
 ((1008, 76), 0.028056003706585734),
 ((1011, 76), 0.020204350359001616),
 ((1012, 76), 0.03594881516886375),
 ((1014, 76), 0.06293149979241842)]

In [39]:
vec.get_feature_names()[1025:1030] # 각 단어 명칭에 접근

['굵직하다', '궁극', '궁금하다', '권고', '권위']

In [40]:
# 데이터프레임으로 예쁘게 만들기
df = pd.DataFrame(list(similarities_sparse.todok().items()), columns=["words", "weight"])

df2 = df.sort_values(by=['weight'], ascending=False) # 단어 페어 간 유사도 높은 순으로 정렬
df2 = df2.reset_index(drop=True)

df3 = df2.loc[np.round(df2['weight']) < 1] # 같은 단어끼리 페어는 유사도 1, 1 미만 페어만 추출
df3 = df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(2827, 2881)",0.499999
1,"(2881, 2827)",0.499999
2,"(2942, 3891)",0.499999
3,"(2942, 2843)",0.499999
4,"(3891, 2942)",0.499999
5,"(2942, 3691)",0.499999
6,"(2843, 2942)",0.499999
7,"(3691, 2942)",0.499999
8,"(3229, 3378)",0.499995
9,"(3378, 3229)",0.499995


## word2vec 기반 연관어 분석
#### 2013년 구글에서 제안, 두 가지 가정 기반
1. 단어의 의미는 그 단어 주변 분포로 이해될 수 있다.
2. 단어의 의미는 단어 벡터 안에 인코딩 될 수 있다.

#### 앞 두개와 다른 점
1. 같은 문맥에서의 1) 출현 빈도 뿐 아니라 2) 단어 위치, 3) 순서에 따라서도 가중치를 다르게 준다.
2. 가중치 산출 후 앞에서와 같이 유사도 공식 이용해 단어간 유사도 산출한다.

#### 두 개 알고리즘
1. CBOW: 주변 단어로 중심단어 예측
2. Skip-gram: 중심 단어로 주변 단어 예측 (window size 따라 반복학습 더 많이 함 -> 더 정확한 경우 많다)

#### 참조
Distributed Representations of Words and Phrases and their Compositionality
(https://arxiv.org/pdf/1310.4546.pdf)

In [66]:
txt = []
for line in whole:
    txt.extend(list(set(line)))

model = Word2Vec(txt, vector_size=100, window = 5, min_count=2, workers=4, sg=1) # skip-gram, window=주변단어, min_count=전체문서에서 최소출현횟수, workers=cpu 코어수
# model.wv.similarity('게임', '규제')
# model.wv.most_similar(positive=["게임"], topn=5)
model.wv.index_to_key

['다',
 '이',
 '0',
 '1',
 '기',
 '2',
 '스',
 '원',
 '%',
 '지',
 '대',
 '억',
 '하',
 '3',
 '시',
 '가',
 '4',
 '리',
 '.',
 '사',
 '보',
 '인',
 '일',
 '6',
 '5',
 '9',
 '비',
 '정',
 '제',
 '자',
 '8',
 '성',
 '7',
 '전',
 '업',
 '로',
 '장',
 '상',
 '해',
 '수',
 '드',
 '트',
 '나',
 '분',
 '한',
 '임',
 '부',
 '게',
 '오',
 '주',
 '계',
 '어',
 '라',
 '국',
 '만',
 '도',
 'P',
 '동',
 '매',
 '경',
 '출',
 '중',
 '영',
 '년',
 '서',
 '아',
 '모',
 '반',
 '위',
 '작',
 '진',
 '용',
 '내',
 '공',
 '르',
 '행',
 '화',
 '개',
 '실',
 '소',
 '고',
 '과',
 '우',
 '확',
 '문',
 '바',
 '신',
 '증',
 'N',
 '유',
 '체',
 '포',
 '적',
 '크',
 '여',
 '치',
 '구',
 '관',
 '간',
 '재',
 '월',
 '프',
 '선',
 '결',
 '표',
 '회',
 '익',
 '히',
 '발',
 '조',
 '연',
 '즈',
 '각',
 '무',
 '규',
 '안',
 '니',
 '레',
 '승',
 '미',
 '단',
 '속',
 '준',
 '금',
 '외',
 '거',
 '예',
 '플',
 '요',
 '입',
 '통',
 '난',
 '목',
 '코',
 '두',
 '최',
 '디',
 'C',
 '데',
 '블',
 '마',
 '방',
 '식',
 '의',
 '감',
 '세',
 '불',
 '늘',
 '투',
 '산',
 '력',
 '타',
 '그',
 '터',
 '추',
 '권',
 '역',
 '차',
 '록',
 'I',
 '배',
 '종',
 '버',
 '현',
 '커',
 '향',
 '머'